In [1]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories
from pathlib import Path

In [9]:
params = read_yaml(PARAMS_FILE_PATH)

[2024-03-26 12:05:23,834: INFO: common: yaml file: params.yaml loaded successfully]


In [14]:
params.OFFSET

-1